# CARGA DE LIBRERIAS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy

from sklearn.model_selection import train_test_split

import torch
from torch import nn  # Modelos neuronales y funciones de Loss

from torch import optim # (3er paso del algoritmo de retropropagación) Optimizadores ---> Gradiente descendiente, Adam, AdaDelta, etc
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Elige el dispositivo. Utilizará GPU si está disponible

# VISUALIZACION DE LOS DATOS

In [ ]:
data = pd.read_csv('irisbin.csv', header=None).to_numpy()

data  # La clase está repartida en 3 columnas (codificación one-hot)

new_data = data[:,:-2].copy()

new_data[:,-1] = data[:,-3:].argmax(axis=1)  # Extraigo la posición del "1" para cada patrón y la guardo al final ---> Clase

### GENERO UNA PROYECCION CON PCA PARA VISUALIZAR LOS DATOS EN 2D

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit(new_data[:,:-1],new_data[:,-1]).transform(new_data[:,:-1])

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(8,8))
ax.scatter(X_pca[:, 0],
           X_pca[:, 1],
           s=100,
           c=[f'{["blue","green","red"][int(i)]}' for i in new_data[:,-1]],
           alpha=0.96,
           lw=1)
plt.grid(True)
plt.xlabel('PC 1', fontsize=14)
plt.ylabel('PC 2', fontsize=14)
plt.grid(True);

# PARTICIONADO DE DATOS

A partir del dataset `irisbin.csv`, genere las particiones de `train`, `validation` y `test` con las proporciones $65$/$15$/$20$ respectivamente. Almacene las particiones en archivos CSV independientes bajo los nombres: `data_trn.csv`, `data_val.csv` y `data_tst.csv`. Los archivos no deben contener cabeceras.

# CONSTRUCCION DEL DATASET

In [ ]:
class dataset(Dataset):
    '''
    Esta clase maneja la lectura de los datos y provee un mecanismo
    para alimentar los modelos con los patrones.
    '''
    
    #===================================================
    def __init__(self, filename):
        
        pass
    
    #===================================================
    def __len__(self):
        '''
        Devuelve el número de patrones en el dataset.
        '''
        pass
    
    
    #===================================================
    def __getitem__(self, idx):
        '''
        Devuelve el/los patrones indicados.
        '''
        pass

# CONSTRUCCION DEL MODELO

## PERCEPTRON MULTICAPA

In [ ]:
class model(nn.Module):

    #====================================
    def __init__(self, n_features, n_inputs, n_outputs):
        '''
        Esta función inicializa/construye el perceptrón.
        n_features: features de cada patrón (2 para OR y XOR)
        n_outputs: cantidad de salidas esperadas.
        '''
        
        super().__init__()
        
        pass

    #====================================
    def forward(self, x):
        '''
        Esta función realiza la pasada hacia adelante.
        '''
        
        pass

# LOOPS

## ENTRENAMIENTO

In [ ]:
def train_step(model, data, loss_function, optimizer, device):
    '''
    Esta función se encarga de pasar todos los patrones
    a través del modelo neuronal y realizar el ajuste de los pesos.
    '''
    
    model.train()  # Calcula gradientes
    
    N_batches = len(data)  # Número de batches = N_patrones/N_patrones_x_batch
    
    error = 0
    
    #==============================================================
    for idx,(X,y) in enumerate(data):

        X = X.to(device)  # Se envían los datos a la GPU (si se dispone)
        y = y.to(device)  # Se envían los datos a la GPU (si se dispone)

        optimizer.zero_grad()  # Se limpia el caché del optimizador
        
        #-----------------------
        # Pasada hacia adelante
        # (Forward pass)
        #-----------------------
        y_pred = model(X)
        
        #---------------------------
        # Cálculo del error (Loss)
        #---------------------------
        if (data.batch_size == 1):
            loss = loss_function(y_pred.squeeze(), y.squeeze())
        else:
            loss = loss_function(y_pred.squeeze(), y)
        
        error += loss.item()
        
        #-----------------------
        # Retropropagación
        # (Backward pass)
        #-----------------------
        loss.backward()  # Calcula los gradientes en cada capa
        optimizer.step()  # Corrige los pesos
    #==============================================================
    
    error /= N_batches
    
    return error, model

## VALIDACION / TEST

In [ ]:
def predict_step(model, data, loss_function, device):
    '''
    Esta función se encarga de pasar todos los patrones
    hacia adelante a través del modelo para generar
    las predicciones.
    '''
    
    model.eval()  # Se desactiva el funcionamiento
                  # de algunos elementos especiales de PyTorch
    
    N_batches = len(data)  # Número de batches = N_patrones/N_patrones_x_batch
    
    error = 0
    
    Y = torch.tensor([])
    Yp = torch.tensor([])
    
    #==============================================================
    with torch.no_grad():  # Se desactiva el cálculo de gradientes
        
        for idx,(X,y) in enumerate(data):
            
            #-------------------------------------------------------------
            # En estas líneas acumulamos las salidas deseadas
            # en un único vector, de manera de tener ordenados
            # los pares "salida deseada" | "salida predicha" para
            # calcular medidas de desempeño al finalizar esta etapa.
            #-------------------------------------------------------------
            if (Y.shape[0] == 0):
                Y = torch.hstack( (Y, y) )
            else:
                Y = torch.vstack( (Y, y) )

            
            X = X.to(device)  # Se envían los datos a la GPU (si se dispone)
            y = y.to(device)  # Se envían los datos a la GPU (si se dispone)

            #-----------------------
            # Pasada hacia adelante
            # (Forward pass)
            #-----------------------
            y_pred = model(X)

            #-------------------------------------------------------------
            # En estas líneas acumulamos las salidas predichas
            # del modelo en un único vector, de manera de tener
            # ordenados los pares "salida deseada" | "salida predicha"
            # para calcular medidas de desempeño al finalizar esta etapa.
            # El método "cpu()" retorna los datos a la CPU en caso de estar en la GPU.
            #-------------------------------------------------------------
            if (Yp.shape[0] == 0):
                Yp = torch.hstack( (Yp, y_pred.cpu()) )
            else:
                Yp = torch.vstack( (Yp, y_pred.cpu()) )

            #---------------------------
            # Cálculo del error (Loss)
            #---------------------------
            if (data.batch_size == 1):
                loss = loss_function(y_pred.squeeze(), y.squeeze())  # El método "squeeze()" elimina todas las dimensiones con valor "1"
                                                                     # Ej. el vector [[1,2,3]] tiene dimensiones (1,3). Luego de aplicar
                                                                     # "squeeze()", el vector resultante [1,2,3] tiene dimensiones (3,)
            else:
                loss = loss_function(y_pred.squeeze(), y)

            error += loss.item()
    #==============================================================
    
    error /= N_batches
    
    #------------------
    
    return error, Y, Yp

# EXPERIMENTO

In [ ]:
#=================================
# Definimos los archivos de datos
#=================================
filename = 'irisbin.csv'

#==========================================
# Inicializamos parámetros del experimento
#==========================================
LEARNING_RATE = None  # Tasa de aprendizaje

MOMENTUM = None  # Término de momento

MAX_EPOCAS = None  # Defino el número máximo de épocas
                  # de entrenamiento.

PATIENCE = None   # Defino el máximo número de épocas
                  # sin mejorar el error de validación
                  # para detener el entrenamiento.

BATCH_SIZE = None  # Número de patrones en cada batch
                   # Se denomina 'batch' a un conjunto de patrones
                   # que se procesan juntos durante una pasada a 
                   # través de la red neuronal  durante el aprendizaje.
                   # Ej. si el batch incluye 10 patrones, los mismos
                   # son usados para realizar los pasos hacia adelante,
                   # la retropropagación y el cálculo de la actualización
                   # de los pesos de acuerdo al error cometido con cada patrón.
                   # Sin embargo, al usar un batch se combinan las actualizaciones
                   # de pesos debidas a cada patrón y se realiza una única
                   # actualización "promedio".

#-------------------------------------------------

acc = 0.  # Inicializo el accuracy inicial
epoca = 0  # Inicializo contador de épocas

MIN_ERROR = 1E6   # Inicializo la variable para
                  # registrar el mínimo error cometido.


#===========================================================
# Construimos los datasets para entrenamiento y validación
#===========================================================
filename_train = 'data_trn.csv'
filename_validation = 'data_val.csv'
filename_test = 'data_tst.csv'

trn = dataset(filename_train)
val = dataset(filename_validation)
tst = dataset(filename_test)


#=============================================================
# Construimos los dataloaders para entrenamiento y validación
#=============================================================
train_data = DataLoader(trn, batch_size=BATCH_SIZE, shuffle=True)
validation_data = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)


#=============================================
# Inicializamos el modelo
#=============================================
modelo = model(...)
modelo.to(device)


#=============================================
# Definimos la función de LOSS a utilizar
#=============================================
loss_function = nn.MSELoss(reduction='mean').to(device)

#=============================================
# Definimos el optimizador a utilizar
# >>> 3er paso del bacpropagation
#=============================================
optimizer = optim.SGD(modelo.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

## ENTRENAMIENTO DEL MODELO

In [ ]:
error_trn = []  # Inicializo estructura para almacenar
                # los errores en el tiempo

error_val = []

accuracy = []  # Inicializo estructura para almacenar
               # el accuracy en el tiempo

STOP = False
counter = 0

best_model = None
best_model_weights = None

#===============================================================
while (epoca < MAX_EPOCAS) and (not STOP):

    epoca += 1
    
    #----------------------
    # ENTRENAMIENTO
    #----------------------
    _,modelo = train_step(modelo, train_data, loss_function, optimizer, device)
    
    #----------------------
    # VALIDACION
    #----------------------
    e_trn,Y_trn,Yp_trn = predict_step(modelo, train_data, loss_function, device)
    e_val,Y_val,Yp_val = predict_step(modelo, validation_data, loss_function, device)

    real_class = torch.argmax(Y_val,dim=1)  # Decodifico la clase ===> ej. [-1, 1, -1] --> 1
    predicted_class = torch.argmax(Yp_val,dim=1)  # Decodifico la clase ===> ej. [-1, -1, 1] --> 2
    acc = torch.sum(predicted_class == real_class)/ len(Y_val)
    
    #----------------------
    # ALMACENO MEDIDAS
    #----------------------
    error_trn.append(e_trn)
    error_val.append(e_val)
    accuracy.append(acc)
    
    #-----------------------------------------------
    # CRITERIO DE CORTE Y ALMACENAMIENTO DEL MODELO
    #-----------------------------------------------
    if (e_val < MIN_ERROR):
        MIN_ERROR = e_val
        counter = 0
        
        #·······················
        # Almaceno el modelo
        #·······················
        best_model = deepcopy(modelo)  # Genero una copia independiente
        best_model_weights = best_model.state_dict()
        
    else:
        counter += 1
        if counter > PATIENCE:
            STOP = True
    
    #--------------------------------------------
    # MUESTRO REPORTE POR PANTALLA (POR EPOCA)
    #--------------------------------------------
    if (epoca % 100) == 0:
        print(f'Epoca: {epoca} -- Error [trn]: {e_trn:.4}\t--\tError [val]: {e_val:.4}\t--\tTasa acierto [val]: {acc:.4}\n')
#===============================================================

#--------------------------------------------
# MUESTRO REPORTE POR PANTALLA (FINAL)
#--------------------------------------------
print('='*120)
print(f'FINAL -- Epoca: {epoca} -- Error [trn]: {e_trn:.4}\t-- Error [val]: {e_val:.4}\t--\tTasa acierto [val]: {acc:.4}')
print('='*120)

#-----------------------------
# GUARDO MEJOR MODELO A DISCO
#-----------------------------
torch.save(best_model,
           'best_model.pt',
           _use_new_zipfile_serialization=True)
        
#----------------------------------------------
# GUARDAMOS LOS PESOS DEL MEJOR MODELO A DISCO
#----------------------------------------------
torch.save(best_model.state_dict(),
           'best_model_state_dict.pt',
           _use_new_zipfile_serialization=True)

## GRAFICAMOS LAS SALIDAS

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(30,6))

epocas = np.arange(epoca)

# ERROR
ax[0].plot(epocas, error_trn, '-r', lw=2, label='Train')
ax[0].plot(epocas, error_val, '-g', lw=2, label='Val')
ax[0].grid(True)
ax[0].legend()
ax[0].set_xlim(0,MAX_EPOCAS)
ax[0].set_xlabel('Epocas', fontsize=12)
ax[0].set_ylabel('CE', fontsize=12)

# ACC
ax[1].plot(epocas, accuracy, '-b', lw=2)
ax[1].grid(True)
ax[1].set_xlim(0,MAX_EPOCAS)
ax[1].set_xlabel('Epocas', fontsize=12)
ax[1].set_ylabel('Acc [val]', fontsize=12);

---

## LECTURA DE DATOS DE EVALUACION

In [ ]:
#=====================================
# LEVANTAMOS DE DISCO EL MEJOR MODELO
#=====================================

del modelo  # Eliminamos de memoria
            # para asegurarnos de usar
            # el modelo guardado en disco

#--------------------------------------
# Modelo completo (archivo binario)
#--------------------------------------
modelo = torch.load('best_model.pt')

#-----------------------
# A partir de los pesos
#-----------------------
#best_model = torch.load('best_model_state_dict.pt')
#modelo = MODELO(n_features=2, n_inputs=2, n_outputs=1)
#modelo.load_state_dict(best_model)
#modelo.to(device)

In [ ]:
#=====================================
# CONSTRUIMOS EL DATASET PARA TEST
#=====================================
test_data = DataLoader(tst, batch_size=BATCH_SIZE, shuffle=False)

#=====================================
# EVALUAMOS EL MODELO ENTRENADO
#=====================================
error,Y,Yp = predict_step(modelo, test_data, loss_function, device)

acc = torch.sum(torch.argmax(Yp,dim=1) == torch.argmax(Y,dim=1))/ len(Y)

print(f'\nTasa acierto [test]: {acc:.4}\n')

## GRAFICAMOS LA CLASIFICACION

In [ ]:
from sklearn.metrics import accuracy_score

fig, ax = plt.subplots(1, 3, figsize=(30,10))

DATA = [trn, val, tst]
partition = ['Train', 'Validation', 'Test']

for idx,dataset in enumerate(DATA):

    data = DataLoader(dataset, batch_size=1, shuffle=False)
    
    e,y,yp = predict_step(modelo, data, loss_function, device)

    Y = torch.argmax(y, dim=1)  # Decodifico la clase ===> ej. [-1, -1, 1] --> 2
    Yp = torch.argmax(yp, dim=1)  # Decodifico la clase ===> ej. [1, -1, -1] --> 0
    
    C = []
    for i in range(len(Y)):
        if (Y[i] == Yp[i]):
            if (Y[i] == 0):
                C.append('blue')
            elif (Y[i] == 1):
                C.append('green')
            if (Y[i] == 2):
                C.append('red')
        else:  # (Y[i] != Yp[i])
            if (Y[i] == 0):
                C.append('magenta')
            elif (Y[i] == 1):
                C.append('lightgreen')
            else:
                C.append('cyan')

    # EXTRAIGO LOS PATRONES
    X = np.array([x[0] for x in data.dataset])

    # PROYECTO LOS PATRONES R4 --> R2
    X_pca = pca.transform(X)

    acc = accuracy_score(Y,Yp)
    
    # CONSTRUYO LOS GRAFICOS
    ax[idx].scatter(X_pca[:,0], X_pca[:,1], 100, C)
    ax[idx].set_xlim(-3.5,4.5)
    ax[idx].set_ylim(-1.5,1.5)
    ax[idx].set_title(f'{partition[idx]} - Acc: {acc:.3}', fontsize=16)
    ax[idx].set_xlabel('PC 1', fontsize=14)
    ax[idx].set_ylabel('PC 2', fontsize=14)
    ax[idx].grid(True)